In [10]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime as dt
import numpy as np
#sending email
import smtplib

In [2]:

#Function to get book title. 
def get_book_title(soup):
    #Try going find the book title first. if can then get book title
    #else, product_title = "None"
    try: 
        product_title = soup.find("span",attrs={"id":'productTitle'}).text.strip()
    except AttributeError:
        product_title = "None"
    return product_title
        


In [3]:
def get_book_author(soup):
    try:
        product_author = soup.find("span",attrs={"class":'author notFaded'}).find("a",attrs={"class":"a-link-normal"}).text.strip()
    except AttributeError:
        product_author = "None"
    return product_author

In [4]:
def get_book_rating(soup):
    try: 
        product_rating = soup.find("i",attrs={"class":'a-icon a-icon-star a-star-4-5 cm-cr-review-stars-spacing-big'}).text.strip()
    except AttributeError:
        product_rating = "None"

    return product_rating

In [5]:
def get_book_price(soup):
    try: 
        product_price = soup.find("div",attrs={"class":'a-section a-spacing-none aok-align-center aok-relative'}).find("span",attrs={"class":'aok-offscreen'}).text.strip()
    except AttributeError:
        product_price = "None"
    return product_price

In [6]:
def get_book_review(soup):
    try: 
        product_review = soup.find("span",attrs={"id":'acrCustomerReviewText'}).text.strip()
    except AttributeError: 
        product_review = "None"
    return product_review 
        

In [8]:
URL = "https://www.amazon.com.au/s?k=books&crid=1EISML3B83NV&sprefix=book%2Caps%2C242&ref=nb_sb_noss_1"
HEADERS = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.1 Safari/605.1.15",
                }
response = requests.get(URL,headers = HEADERS)
soup = BeautifulSoup(response.content,"html.parser")
pretty_soup = BeautifulSoup(soup.prettify(),"html.parser")

links = pretty_soup.find_all("a",attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
list_of_books_links =[]
for book_link in links:
    list_of_books_links.append(book_link.get('href'))


book_data_dictionaries = {"Title":[],"Author":[],"Rating":[],"Review":[], "Price":[]}
for link in list_of_books_links:
     #new_link = "https://www.amazon.com.au/"+link
    new_book = requests.get("https://www.amazon.com.au/"+link,headers= HEADERS)
    new_soup = BeautifulSoup(new_book.content,"html.parser")
    
    #Calling function to get book details 
    book_title = get_book_title(new_soup)
    book_author = get_book_author(new_soup)
    book_rating = get_book_rating(new_soup)
    book_review = get_book_review(new_soup)
    book_price = get_book_price(new_soup)
    #adding books details to the dictionaries
    book_data_dictionaries["Title"].append(book_title)
    book_data_dictionaries["Author"].append(book_author)
    book_data_dictionaries["Rating"].append(book_rating)
    book_data_dictionaries["Review"].append(book_review)
    book_data_dictionaries["Price"].append(book_price)
     #print(book_title)


    

In [15]:
books_dataframe = pd.DataFrame.from_dict(book_data_dictionaries)
books_dataframe["Title"].replace("None",np.nan,inplace = True)
books_dataframe = books_dataframe.dropna()
books_dataframe.to_csv("books.csv")

In [13]:
books_dataframe

<bound method DataFrame.dropna of                                                 Title             Author  \
0   Spelling for Life: Uncovering the Simplicity a...          Lyn Stone   
1   The Art of Game Design: A Book of Lenses, Thir...       Jesse Schell   
2            Let it go: Emotions are Energy in Motion     Craig Phillips   
3   The Housemaid: An absolutely addictive psychol...    Freida McFadden   
4   King of Sloth: A Forced Proximity Romance (Kin...          Ana Huang   
..                                                ...                ...   
56                        Modern Recording Techniques  David Miles Huber   
57                           Max Cyber Terror Revenge     Anthony Silman   
58  Claris: Fashion Show Fiasco: The Chicest Mouse...         Megan Hess   
59  Solving Product Design Exercises: Questions & ...   Artiom Dashinsky   
60                                                NaN               None   

                Rating           Review              